# LangChain サンプル 2: Retrieval

### PDF ファイルをロードする DocumentLoader やベクターデータベースなどのインストール

In [ ]:
!pip install pymupdf

In [ ]:
!pip install langchain-text-splitters

In [ ]:
!pip install langchain-chroma==0.1.4

### DocumentLoader と　Embed モデルによるベクターデータベース作成のサンプル

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader 
from langchain_aws import BedrockEmbeddings 
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma 

loader = PyMuPDFLoader("./AnyCompany.pdf") # PDF ファイルを読み込み
documents = loader.load()

text_splitter = CharacterTextSplitter(  # ドキュメント分割用に CharacterTextSplitterを初期化
    chunk_size=300,
    chunk_overlap=0
)
splitted_documents = text_splitter.split_documents(documents)

embeddings = BedrockEmbeddings( # BedrockEmbeddings を初期化
    model_id = "amazon.titan-embed-text-v1" 
)

database = Chroma.from_documents(splitted_documents,embeddings)

print("データベースの作成が完了しました。") 

### ベクターデータベースから類似性の高いドキュメントを取得するサンプル

In [ ]:
documents = database.similarity_search("社員が結婚したときにの休暇は何日？") # データベースから類似度の高いドキュメントを取得
print(f"ドキュメントの数: {len(documents)}") # ドキュメントの数を表示

for document in documents:
    print(f"ドキュメントの内容: {document.page_content}") # ドキュメントの内容を表示

### ベクターデータベースから類似性の高いドキュメントを取得してモデルへ問い合わせを行うサンプル

In [ ]:
from langchain_aws import ChatBedrock  
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages.human import HumanMessage 

query = "社員が結婚したときにの休暇は何日？"

documents = database.similarity_search(query)

documents_string = "" # ドキュメントの内容を格納する変数を初期化

for document in documents:
    documents_string += f"""
---------------------------
{document.page_content}
""" # ドキュメントの内容を追加

prompt = PromptTemplate( # PromptTemplateを初期化
    template="""文章を元に質問に答えてください。 

文章: 
{document}

質問: {query}
""",
    input_variables=["document","query"] # 入力変数を指定
)

chat = ChatBedrock( #  ChatBedrockを初期化
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 
)

result = chat.invoke([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])

print(result.content)

### ベクターデータベースから類似性の高いドキュメントを取得して　RetrievalQA を使用してモデルへ問い合わせを行うサンプル

In [ ]:
from langchain.chains import RetrievalQA  

retriever = database.as_retriever() # データベースをRetrieverに変換

qa = RetrievalQA.from_llm(         #  RetrievalQAを初期化
    llm=chat,                      #  Chat modelsを指定
    retriever=retriever,           #  Retrieverを指定
    return_source_documents=True   # レスポンスにソースドキュメントを含めるかを指定
)


result = qa.invoke("社員が結婚したときにの休暇は何日？")

print(result["result"]) # レスポンスを表示

print(result["source_documents"]) # ソースドキュメントを表示

### Retriever 課題 : AnyCompany 社の決算説明資料のベクターデータベースを作成して業績について問い合わせてみよう
1. AnyCompany 社の決算説明資料 AnyCompany_IR.pdf からベクターデータベースを作成します。
1. RetrievalQA を使用して、AnyCompany社の営業利益がいくらかを問い合わせてください。出力は英語でもかまいません。
- **上記は、セルを分けてもかまいません。**